In [1]:
import numpy as np 

import torch

from pytorch_tabnet.tab_model import TabNetRegressor
from pytorch_tabnet.metrics import Metric
from scipy.stats import spearmanr

In [2]:
class spearmanr_metric(Metric):
    """
    Spearman correlation metric
    """

    def __init__(self):
        self._name = "spearmanr"
        self._maximize = True

    def __call__(self, y_true, y_score):
        """
        Compute Spearman r of predictions.

        Parameters
        ----------
        y_true: np.ndarray
            Target matrix or vector
        y_score: np.ndarray
            Score matrix or vector

        Returns
        -------
            float
            Spearman r of predictions vs targets.
        """
        return spearmanr(y_score, y_true).statistic

def spearmanr_loss_fn(y_pred, y_true):
    """
    Dummy example similar to using default torch.nn.functional.cross_entropy
    """
    num = torch.sum((y_pred - torch.mean(y_pred)) * (y_true - torch.mean(y_true)))
    den = torch.norm(y_pred - torch.mean(y_pred)) * torch.norm(y_true - torch.mean(y_true))
    
    return 1 - num/den

In [3]:
n_chunks = 5

clf = TabNetRegressor()

chunk_num = 0
warm_start = False

for train_chunk_num in range(1, n_chunks):
    train_X = np.load(f"train_X_{train_chunk_num}.npy")
    train_Y = np.load(f"train_Y_{train_chunk_num}.npy")
    for val_chunk_num in range(1, n_chunks):

        print("TRAIN_NUM: ", train_chunk_num, ", VAL_NUM: ", val_chunk_num)

        val_X = np.load(f"val_X_{val_chunk_num}.npy")
        val_Y = np.load(f"val_Y_{val_chunk_num}.npy")
        
        clf.fit(
            train_X.reshape(train_X.shape[0], -1), train_Y.reshape(-1, 1),
            eval_set=[(val_X.reshape(val_X.shape[0], -1), val_Y.reshape(-1, 1))],
            eval_metric=['spearmanr', spearmanr_metric],
            loss_fn=spearmanr_loss_fn,
            batch_size=1024, virtual_batch_size=128,
            max_epochs=50 , patience=10,
            drop_last=False, warm_start=warm_start,
        )
    
        warm_start = True

# save tabnet model
saving_path_name = "./tabnet_model_test_1"
saved_filepath = clf.save_model(saving_path_name)

/home/parashara/micromamba/envs/novozymes/lib/python3.11/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


TRAIN_NUM:  1 , VAL_NUM:  1
epoch 0  | loss: 0.98693 | val_0_spearmanr: 0.02345 |  0:00:03s
epoch 1  | loss: 0.99182 | val_0_spearmanr: 0.00326 |  0:00:06s
epoch 2  | loss: 0.97326 | val_0_spearmanr: 0.01664 |  0:00:09s
epoch 3  | loss: 0.96265 | val_0_spearmanr: -0.00934|  0:00:13s
epoch 4  | loss: 0.95075 | val_0_spearmanr: -0.01168|  0:00:16s
epoch 5  | loss: 0.94479 | val_0_spearmanr: 0.0433  |  0:00:19s
epoch 6  | loss: 0.9507  | val_0_spearmanr: 0.02598 |  0:00:22s
epoch 7  | loss: 0.9416  | val_0_spearmanr: -0.02945|  0:00:25s
epoch 8  | loss: 0.97185 | val_0_spearmanr: -0.03285|  0:00:28s
epoch 9  | loss: 0.93883 | val_0_spearmanr: -0.00203|  0:00:31s
epoch 10 | loss: 0.81317 | val_0_spearmanr: -0.01054|  0:00:34s
epoch 11 | loss: 0.79402 | val_0_spearmanr: -0.01527|  0:00:37s
epoch 12 | loss: 0.81463 | val_0_spearmanr: -0.00955|  0:00:40s
epoch 13 | loss: 0.78666 | val_0_spearmanr: 0.02664 |  0:00:43s
epoch 14 | loss: 0.7659  | val_0_spearmanr: -0.03208|  0:00:47s
epoch 15 | l

/home/parashara/micromamba/envs/novozymes/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


TRAIN_NUM:  1 , VAL_NUM:  2
epoch 0  | loss: 0.97367 | val_0_spearmanr: 0.03336 |  0:00:03s
epoch 1  | loss: 0.95495 | val_0_spearmanr: 0.00431 |  0:00:06s
epoch 2  | loss: 0.94104 | val_0_spearmanr: 0.06734 |  0:00:09s
epoch 3  | loss: 0.87406 | val_0_spearmanr: 0.0699  |  0:00:12s
epoch 4  | loss: 0.94377 | val_0_spearmanr: 0.00573 |  0:00:15s
epoch 5  | loss: 0.96111 | val_0_spearmanr: 0.04524 |  0:00:18s
epoch 6  | loss: 0.91848 | val_0_spearmanr: 0.05543 |  0:00:21s
epoch 7  | loss: 0.89217 | val_0_spearmanr: 0.01172 |  0:00:24s
epoch 8  | loss: 0.80836 | val_0_spearmanr: -0.0089 |  0:00:28s
epoch 9  | loss: 0.79394 | val_0_spearmanr: 0.02695 |  0:00:31s
epoch 10 | loss: 0.77219 | val_0_spearmanr: 0.07131 |  0:00:34s
epoch 11 | loss: 0.7912  | val_0_spearmanr: 0.06087 |  0:00:37s
epoch 12 | loss: 0.7732  | val_0_spearmanr: 0.08622 |  0:00:40s
epoch 13 | loss: 0.78778 | val_0_spearmanr: 0.08745 |  0:00:43s
epoch 14 | loss: 0.78226 | val_0_spearmanr: 0.0529  |  0:00:46s
epoch 15 | l

/home/parashara/micromamba/envs/novozymes/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


...

TRAIN_NUM:  4 , VAL_NUM:  3
epoch 0  | loss: 0.28342 | val_0_spearmanr: 0.33256 |  0:00:05s
epoch 1  | loss: 0.27198 | val_0_spearmanr: 0.27499 |  0:00:11s
epoch 2  | loss: 0.25675 | val_0_spearmanr: 0.33068 |  0:00:17s
epoch 3  | loss: 0.24512 | val_0_spearmanr: 0.26895 |  0:00:23s
epoch 4  | loss: 0.24167 | val_0_spearmanr: 0.34228 |  0:00:29s
epoch 5  | loss: 0.22869 | val_0_spearmanr: 0.30433 |  0:00:35s
epoch 6  | loss: 0.22538 | val_0_spearmanr: 0.29057 |  0:00:40s
epoch 7  | loss: 0.22749 | val_0_spearmanr: 0.30121 |  0:00:46s
epoch 8  | loss: 0.21292 | val_0_spearmanr: 0.30483 |  0:00:52s
epoch 9  | loss: 0.21249 | val_0_spearmanr: 0.3181  |  0:00:58s
epoch 10 | loss: 0.21191 | val_0_spearmanr: 0.31735 |  0:01:04s
epoch 11 | loss: 0.2119  | val_0_spearmanr: 0.31162 |  0:01:10s
epoch 12 | loss: 0.20908 | val_0_spearmanr: 0.336   |  0:01:15s
epoch 13 | loss: 0.20406 | val_0_spearmanr: 0.3166  |  0:01:21s
epoch 14 | loss: 0.20035 | val_0_spearmanr: 0.322   |  0:01:27s

Early stopp

/home/parashara/micromamba/envs/novozymes/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


TRAIN_NUM:  4 , VAL_NUM:  4
epoch 0  | loss: 0.25614 | val_0_spearmanr: 0.347   |  0:00:06s
epoch 1  | loss: 0.24283 | val_0_spearmanr: 0.33794 |  0:00:12s
epoch 2  | loss: 0.23003 | val_0_spearmanr: 0.34997 |  0:00:18s
epoch 3  | loss: 0.22575 | val_0_spearmanr: 0.34116 |  0:00:24s
epoch 4  | loss: 0.21828 | val_0_spearmanr: 0.35122 |  0:00:31s
epoch 5  | loss: 0.21588 | val_0_spearmanr: 0.34667 |  0:00:37s
epoch 6  | loss: 0.20382 | val_0_spearmanr: 0.35222 |  0:00:43s
epoch 7  | loss: 0.20589 | val_0_spearmanr: 0.35443 |  0:00:49s
epoch 8  | loss: 0.19999 | val_0_spearmanr: 0.35695 |  0:00:55s
epoch 9  | loss: 0.1915  | val_0_spearmanr: 0.32988 |  0:01:01s
epoch 10 | loss: 0.19319 | val_0_spearmanr: 0.36075 |  0:01:08s
epoch 11 | loss: 0.1939  | val_0_spearmanr: 0.35721 |  0:01:14s
epoch 12 | loss: 0.1874  | val_0_spearmanr: 0.34805 |  0:01:20s
epoch 13 | loss: 0.18604 | val_0_spearmanr: 0.3484  |  0:01:26s
epoch 14 | loss: 0.1837  | val_0_spearmanr: 0.34523 |  0:01:32s
epoch 15 | l

/home/parashara/micromamba/envs/novozymes/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at ./tabnet_model_test_1.zip


In [4]:
n_chunks = 5

chunk_num = 0

for train_chunk_num in np.random.permutation(range(1, n_chunks)):
    train_X = np.load(f"train_X_{train_chunk_num}.npy")
    train_Y = np.load(f"train_Y_{train_chunk_num}.npy")
    
    for val_chunk_num in np.random.permutation(range(1, n_chunks)):

        print("TRAIN_NUM: ", train_chunk_num, ", VAL_NUM: ", val_chunk_num)

        val_X = np.load(f"val_X_{val_chunk_num}.npy")
        val_Y = np.load(f"val_Y_{val_chunk_num}.npy")
        
        clf.fit(
            train_X.reshape(train_X.shape[0], -1), train_Y.reshape(-1, 1),
            eval_set=[(val_X.reshape(val_X.shape[0], -1), val_Y.reshape(-1, 1))],
            eval_metric=['spearmanr', spearmanr_metric],
            loss_fn=spearmanr_loss_fn,
            batch_size=1024, virtual_batch_size=128,
            max_epochs=50 , patience=10,
            drop_last=False, warm_start=True,
        )
    
# save tabnet model
saving_path_name = "./tabnet_model_test_2"
saved_filepath = clf.save_model(saving_path_name)

TRAIN_NUM:  4 , VAL_NUM:  2
epoch 0  | loss: 0.21373 | val_0_spearmanr: 0.34241 |  0:00:05s
epoch 1  | loss: 0.20173 | val_0_spearmanr: 0.3207  |  0:00:11s
epoch 2  | loss: 0.19663 | val_0_spearmanr: 0.35533 |  0:00:17s
epoch 3  | loss: 0.19178 | val_0_spearmanr: 0.35332 |  0:00:22s
epoch 4  | loss: 0.18888 | val_0_spearmanr: 0.34361 |  0:00:28s
epoch 5  | loss: 0.1859  | val_0_spearmanr: 0.35847 |  0:00:34s
epoch 6  | loss: 0.18697 | val_0_spearmanr: 0.33475 |  0:00:40s
epoch 7  | loss: 0.18206 | val_0_spearmanr: 0.33783 |  0:00:46s
epoch 8  | loss: 0.17869 | val_0_spearmanr: 0.34326 |  0:00:51s
epoch 9  | loss: 0.17957 | val_0_spearmanr: 0.3395  |  0:00:57s
epoch 10 | loss: 0.17571 | val_0_spearmanr: 0.34456 |  0:01:03s
epoch 11 | loss: 0.17451 | val_0_spearmanr: 0.32147 |  0:01:09s
epoch 12 | loss: 0.17436 | val_0_spearmanr: 0.3387  |  0:01:14s
epoch 13 | loss: 0.17218 | val_0_spearmanr: 0.3473  |  0:01:20s
epoch 14 | loss: 0.16816 | val_0_spearmanr: 0.32698 |  0:01:26s
epoch 15 | l

/home/parashara/micromamba/envs/novozymes/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


TRAIN_NUM:  4 , VAL_NUM:  3
epoch 0  | loss: 0.19585 | val_0_spearmanr: 0.31855 |  0:00:05s
epoch 1  | loss: 0.19041 | val_0_spearmanr: 0.33144 |  0:00:11s
epoch 2  | loss: 0.1907  | val_0_spearmanr: 0.32718 |  0:00:17s
epoch 3  | loss: 0.18383 | val_0_spearmanr: 0.31521 |  0:00:23s
epoch 4  | loss: 0.17644 | val_0_spearmanr: 0.32796 |  0:00:29s
epoch 5  | loss: 0.17497 | val_0_spearmanr: 0.31668 |  0:00:34s
epoch 6  | loss: 0.1748  | val_0_spearmanr: 0.32069 |  0:00:40s
epoch 7  | loss: 0.17272 | val_0_spearmanr: 0.32712 |  0:00:46s
epoch 8  | loss: 0.17303 | val_0_spearmanr: 0.30226 |  0:00:52s
epoch 9  | loss: 0.174   | val_0_spearmanr: 0.2872  |  0:00:58s
epoch 10 | loss: 0.17188 | val_0_spearmanr: 0.31397 |  0:01:04s
epoch 11 | loss: 0.16855 | val_0_spearmanr: 0.29519 |  0:01:09s

Early stopping occurred at epoch 11 with best_epoch = 1 and best_val_0_spearmanr = 0.33144


/home/parashara/micromamba/envs/novozymes/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


TRAIN_NUM:  4 , VAL_NUM:  1
epoch 0  | loss: 0.20181 | val_0_spearmanr: 0.29948 |  0:00:05s
epoch 1  | loss: 0.19297 | val_0_spearmanr: 0.3099  |  0:00:11s
epoch 2  | loss: 0.18715 | val_0_spearmanr: 0.28596 |  0:00:17s
epoch 3  | loss: 0.18288 | val_0_spearmanr: 0.28324 |  0:00:23s
epoch 4  | loss: 0.1813  | val_0_spearmanr: 0.31041 |  0:00:29s
epoch 5  | loss: 0.17315 | val_0_spearmanr: 0.30145 |  0:00:34s
epoch 6  | loss: 0.1731  | val_0_spearmanr: 0.30617 |  0:00:40s
epoch 7  | loss: 0.16992 | val_0_spearmanr: 0.29896 |  0:00:46s
epoch 8  | loss: 0.16921 | val_0_spearmanr: 0.29282 |  0:00:52s
epoch 9  | loss: 0.17047 | val_0_spearmanr: 0.30728 |  0:00:58s
epoch 10 | loss: 0.1674  | val_0_spearmanr: 0.28894 |  0:01:04s
epoch 11 | loss: 0.1669  | val_0_spearmanr: 0.29874 |  0:01:09s
epoch 12 | loss: 0.16483 | val_0_spearmanr: 0.28466 |  0:01:15s
epoch 13 | loss: 0.16413 | val_0_spearmanr: 0.27433 |  0:01:21s
epoch 14 | loss: 0.16091 | val_0_spearmanr: 0.28738 |  0:01:27s

Early stopp

...

TRAIN_NUM:  3 , VAL_NUM:  3
epoch 0  | loss: 0.28215 | val_0_spearmanr: 0.31166 |  0:00:03s
epoch 1  | loss: 0.2513  | val_0_spearmanr: 0.30165 |  0:00:06s
epoch 2  | loss: 0.24091 | val_0_spearmanr: 0.31594 |  0:00:09s
epoch 3  | loss: 0.22682 | val_0_spearmanr: 0.31091 |  0:00:12s
epoch 4  | loss: 0.21974 | val_0_spearmanr: 0.31417 |  0:00:15s
epoch 5  | loss: 0.21176 | val_0_spearmanr: 0.34012 |  0:00:19s
epoch 6  | loss: 0.20181 | val_0_spearmanr: 0.34202 |  0:00:22s
epoch 7  | loss: 0.19413 | val_0_spearmanr: 0.33845 |  0:00:25s
epoch 8  | loss: 0.18891 | val_0_spearmanr: 0.31359 |  0:00:28s
epoch 9  | loss: 0.18601 | val_0_spearmanr: 0.3031  |  0:00:31s
epoch 10 | loss: 0.17669 | val_0_spearmanr: 0.31332 |  0:00:34s
epoch 11 | loss: 0.17113 | val_0_spearmanr: 0.31961 |  0:00:37s
epoch 12 | loss: 0.17086 | val_0_spearmanr: 0.32004 |  0:00:40s
epoch 13 | loss: 0.17024 | val_0_spearmanr: 0.32022 |  0:00:43s
epoch 14 | loss: 0.16532 | val_0_spearmanr: 0.31688 |  0:00:46s
epoch 15 | l

/home/parashara/micromamba/envs/novozymes/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


TRAIN_NUM:  3 , VAL_NUM:  2
epoch 0  | loss: 0.20737 | val_0_spearmanr: 0.34956 |  0:00:03s
epoch 1  | loss: 0.19841 | val_0_spearmanr: 0.34286 |  0:00:06s
epoch 2  | loss: 0.1887  | val_0_spearmanr: 0.34398 |  0:00:09s
epoch 3  | loss: 0.18309 | val_0_spearmanr: 0.35127 |  0:00:12s
epoch 4  | loss: 0.17696 | val_0_spearmanr: 0.3557  |  0:00:15s
epoch 5  | loss: 0.17871 | val_0_spearmanr: 0.35859 |  0:00:18s
epoch 6  | loss: 0.17101 | val_0_spearmanr: 0.35404 |  0:00:21s
epoch 7  | loss: 0.16494 | val_0_spearmanr: 0.34686 |  0:00:24s
epoch 8  | loss: 0.16082 | val_0_spearmanr: 0.34972 |  0:00:27s
epoch 9  | loss: 0.15919 | val_0_spearmanr: 0.35463 |  0:00:30s
epoch 10 | loss: 0.15474 | val_0_spearmanr: 0.35097 |  0:00:33s
epoch 11 | loss: 0.1506  | val_0_spearmanr: 0.35006 |  0:00:36s
epoch 12 | loss: 0.14982 | val_0_spearmanr: 0.34164 |  0:00:40s
epoch 13 | loss: 0.1467  | val_0_spearmanr: 0.3387  |  0:00:43s
epoch 14 | loss: 0.14579 | val_0_spearmanr: 0.35918 |  0:00:46s
epoch 15 | l

/home/parashara/micromamba/envs/novozymes/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at ./tabnet_model_test_2.zip
